# [14]Get_Clean_VacantSet_for_Baltimore_City
--------------------------------------------------------------------
### Get real data

In [182]:
import pandas as pd
vacantprop = pd.read_csv('VacantProps.csv',header = None) # create header later
vacantprop[:2]

,0,1,2
0,Full Address,BLT,Vacant
1,1824 W FAYETTE ST BALTIMORE | MD 21223-1620,0x00B2000000202D00000020,0


### Preprocess

In [183]:
##### drop if the city is not BALTIMORE

print "original # of rows : " , len(vacantprop)
vacantprop = vacantprop[vacantprop[0].str.find("BALTIMORE") != -1]
print "current # of rows : " , len(vacantprop)

##### drop if not vacant 
print "original # of rows : " , len(vacantprop)
vacantprop = vacantprop[vacantprop[2] == '1']
print "current # of rows : " , len(vacantprop)

##### drop duplicate
print "original # of rows : ", len(vacantprop)
vacantprop = vacantprop.drop_duplicates()
print "current # of rows : ", len(vacantprop)

original # of rows :  192037
current # of rows :  179034
original # of rows :  179034
current # of rows :  8681
original # of rows :  8681
current # of rows :  8602


### Get zipcode

In [184]:
vacantprop[:1]

,0,1,2
6,3838 REISTERSTOWN RD BALTIMORE | MD 21215-7616,0x00690C0000201400000020,1


In [185]:
#zip and zip+4
vacantprop = pd.concat([vacantprop,   
                vacantprop[0].str[-4:], 
                vacantprop[0].str[-10:-5]
               ], axis=1)

# rename columns
vacantprop.columns = ["Full Address", "BLT", "Vacant", "zip+4", "zip"]
vacantprop[:1]

,Full Address,BLT,Vacant,zip+4,zip
6,3838 REISTERSTOWN RD BALTIMORE | MD 21215-7616,0x00690C0000201400000020,1,7616,21215


### Get Street Address and House number

In [186]:
def readstreet(stname):
    count = 0
    street = ""
    housenumber = "" 
    for i in xrange(len(stname)):
        if stname[i].isdigit():
            continue
        elif stname[i] == " ":
            housenumber = stname[0:i]
            count = i+1
            break
        elif i == 0:
            print "start from char : " , stname[i]
            break
        else:
            continue
    for i in xrange(count,len(stname)):
        if stname[i] == "B" and stname[i:i+11] == "BALTIMORE |":
            street = stname[count:i-1]
            break
    
    return street, housenumber

In [187]:
streets = []
housenumbers = []
for i in vacantprop.index:
    street,housenumber = readstreet(vacantprop["Full Address"][i])
    streets.append(street)
    housenumbers.append(housenumber)

In [188]:
# append strret, housenumber into columns
vacantprop = vacantprop.assign(street = streets)
vacantprop = vacantprop.assign(housenumber = housenumbers)
vacantprop[:1]

,Full Address,BLT,Vacant,zip+4,zip,street,housenumber
6,3838 REISTERSTOWN RD BALTIMORE | MD 21215-7616,0x00690C0000201400000020,1,7616,21215,REISTERSTOWN RD,3838


In [190]:
#### Check Format in zip and zip+4

#### if zip is not digit ( ex. | MD )
for i in vacantprop[vacantprop.zip.str.isdigit() != True ].index:
    # update zip+4, and zip
    vacantprop.loc[i,"zip+4"] = 0
    vacantprop.loc[i,"zip"] = vacantprop["Full Address"][i][-5:]

In [193]:
vacantprop[vacantprop["zip+4"] == 0][:1]

,Full Address,BLT,Vacant,zip+4,zip,street,housenumber
356,3913-15 DUVALL AVE BALTIMORE | MD 21216,0x00230B0000201800000020,1,0,21216,DUVALL AVE,3913-15


### Check the name set

In [194]:
realvacant = vacantprop

In [195]:
# same as above
realvacant[(realvacant.street.str[-3:] != "AVE") & #Avenue
           (realvacant.street.str[-3:] != " RD") & #Road
           (realvacant.street.str[-3:] != " ST") & #Street
           #(realvacant.street.str[-3:] != " PL") & #Place 
           (realvacant.street.str.find("BENNETT PL")==-1) &
           (realvacant.street.str.find("HILLDALE PL")==-1) &
           (realvacant.street.str.find("COTWOOD PL")==-1) &
           (realvacant.street.str.find("ESTHER PL")==-1) &
           (realvacant.street.str.find("EUTAW PL")==-1) &
           (realvacant.street.str.find("HARDING PL")==-1) &
           (realvacant.street.str.find("SANFORD PL")==-1) &
           (realvacant.street.str.find("PRENTISS PL")==-1) &
           #(realvacant.street.str.find("KUPER PL")==-1) & # Kuper Street 
           (realvacant.street.str.find("RANKIN PL")==-1) &
           (realvacant.street.str.find("ALLEGANY PL")==-1) & # not in OSM
           (realvacant.street.str.find("NASCO PL")==-1) &
           (realvacant.street.str.find("ROCHESTER PL")==-1) &
           (realvacant.street.str.find("CENTRE PL")==-1) &
           (realvacant.street.str.find("PITMAN PL")==-1) &
           (realvacant.street.str.find("JUNEAU PL")==-1) & # not in OSM
           (realvacant.street.str.find("OAKHURST PL")==-1) &
           (realvacant.street.str[-4:] != "BLVD")&  #boulevard
           (realvacant.street.str[-4:] != "MALL")&  #Mall?
           
           (realvacant.street.str[-3:] != " DR") &  #Drive
           (realvacant.street.str[-3:] != " LN") &  #Lane
           (realvacant.street.str[-4:] != " TER") & #Terrace
           (realvacant.street.str[-5:] != " PKWY") & #Parkway
           (realvacant.street.str[-9:] != " DRIVEWAY") &
           (realvacant.street.str[-4:] != " HWY") & # Highway
           (realvacant.street.str[-4:] != " WAY") & # Way
           (realvacant.street.str[-3:] != " CT") & # Court
           (realvacant.street.str[-2:] != " S") & # South
           (realvacant.street.str[-2:] != " N") & # North
           
           (realvacant.street.str[-4:] != " CIR") & # Circle
           (realvacant.street.str[-5:] != " WALK") & # Walk
           (realvacant.street.str[-3:] != " SQ") & # Square
           
           
           (realvacant.street.str[-7:] != "JUNEWAY") & 
           (realvacant.street.str[-12:] != "THE ALAMEDA") &
           (realvacant.street.str[-15:] != "LOYOLA SOUTHWAY") &
           (realvacant.street.str[-10:] != "N BROADWAY") & # North Broadway
           (realvacant.street.str[-10:] != "S BROADWAY") & # South Broadway
           (realvacant.street.str[-15:] != "LOYOLA NORTHWAY") &
           (realvacant.street.str[-16:] != "DUKELAND ST REAR") & # North Dukeland Street
           (realvacant.street.str[-18:] != "SPAULDING AVE REAR") & # Spaulding Avenue
           (realvacant.street.str[-18:] != "CATHEDRAL ST STE 6") & # Cathedral Street 
           (realvacant.street.str[-21:] != "REISTERSTOWN RD APT 2") & # REISTERSTOWN Road
           (realvacant.street.str[-16:] != "E 22ND ST APT 3") # East 22nd Street
          ]

,Full Address,BLT,Vacant,zip+4,zip,street,housenumber
53704,1412 KUPER PL BALTIMORE | MD 21223-3128,0x0008010000205800000020,1,3128,21223,KUPER PL,1412
70119,3100 ARTABAN PL BALTIMORE | MD 21216,0x00F50B000042E200000020,1,0,21216,ARTABAN PL,3100
100543,1304 KUPER PL BALTIMORE | MD 21223-3126,0x0009010000206200000020,1,3126,21223,KUPER PL,1304
100640,1403 KUPER PL BALTIMORE | MD 21223-3129,0x0008010000205000000020,1,3129,21223,KUPER PL,1403
179440,1309 KUPER PL BALTIMORE | MD 21223-3127,0x0009010000205B00000020,1,3127,21223,KUPER PL,1309
187913,1306 KUPER PL BALTIMORE | MD 21223-3126,0x0009010000206100000020,1,3126,21223,KUPER PL,1306


### Replace Street Name in OpenStreetMap format

In [196]:
def renamestreet(stname):
    # as long as there is no " N N " in the string, this function will work well
    stname = " " + stname + " "
    
    # special
    stname = stname.replace(" KUPER PL ", " Kuper Street ")
    stname = stname.replace(" DUKELAND ST REAR ", " North Dukeland Street ")
    stname = stname.replace(" SPAULDING AVE REAR ", " Spaulding Avenue ")
    stname = stname.replace(" CATHEDRAL ST STE 6 ", " Cathedral Street ")
    stname = stname.replace(" REISTERSTOWN RD APT 2 ", " REISTERSTOWN Road ")
    stname = stname.replace(" E 22ND ST APT 3 ", " East 22nd Street ")
    
    # direction
    stname = stname.replace(" N ", " North ")
    stname = stname.replace(" E ", " East ")
    stname = stname.replace(" W ", " West ")
    stname = stname.replace(" S ", " South ")
    
    # street full name
    stname = stname.replace(" AVE ", " Avenue ")
    stname = stname.replace(" RD ", " Road ")
    stname = stname.replace(" ST ", " Street ")
    stname = stname.replace(" PL ", " Place ")
    stname = stname.replace(" BLVD ", " Boulevard ")
    stname = stname.replace(" DR ", " Drive ")
    stname = stname.replace(" LN ", " Lane ")
    stname = stname.replace(" Ter ", " Terrace ")
    stname = stname.replace(" PKWY ", " Parkway ")
    stname = stname.replace(" HWY ", " Highway ")
    stname = stname.replace(" CT ", " Court ")
    stname = stname.replace(" CIR ", " Circle ")
    stname = stname.replace(" SQ ", " Square ")
        
    return stname


In [199]:
streets = []
housenumbers = []
for i in realvacant.index:
    street,housenumber = readstreet(realvacant["Full Address"][i])
    street = renamestreet(street)
    street = street.title()
    streets.append(street)
    housenumbers.append(housenumber)

In [200]:
realvacant = realvacant.assign(street = streets)
realvacant = realvacant.assign(housenumber = housenumbers)

#### Result

In [204]:
print "# of vacant houses in Baltimore : " , len(realvacant)
realvacant

# of vacant houses in Baltimore :  8602


,Full Address,BLT,Vacant,zip+4,zip,street,housenumber
6,3838 REISTERSTOWN RD BALTIMORE | MD 21215-7616,0x00690C0000201400000020,1,7616,21215,Reisterstown Road,3838
18,3807 PARK HEIGHTS AVE BALTIMORE | MD 21215-7611,0x000D0D0000200400000020,1,7611,21215,Park Heights Avenue,3807
25,4802 PALMER AVE BALTIMORE | MD 21215-6006,0x0008120000206A00000020,1,6006,21215,Palmer Avenue,4802
53,2602 LOYOLA SOUTHWAY BALTIMORE | MD 21215-7029,0x00130D0000440200000020,1,7029,21215,Loyola Southway,2602
68,3007 OAKLEY AVE BALTIMORE | MD 21215-6019,0x0008120000204E00000020,1,6019,21215,Oakley Avenue,3007
70,2622 LEHMAN ST BALTIMORE | MD 21223-2809,0x005C080000203E00000020,1,2809,21223,Lehman Street,2622
84,1420 N LUZERNE AVE BALTIMORE | MD 21213-3718,0x00E1050000201C00000020,1,3718,21213,North Luzerne Avenue,1420
110,719 N LONGWOOD ST BALTIMORE | MD 21216-4629,0x00B1090000202600000047,1,4629,21216,North Longwood Street,719
128,2519 KEYWORTH AVE BALTIMORE | MD 21215-7648,0x000C0D0000201500000020,1,7648,21215,Keyworth Avenue,2519
133,4900 PALMER AVE BALTIMORE | MD 21215-6008,0x0004120000201700000020,1,6008,21215,Palmer Avenue,4900
